# 📈 CHANGE POINT DETECTION

**Mục tiêu:** Phát hiện điểm gãy cấu trúc (Structural Break) trong dữ liệu điểm thi, tập trung vào năm 2025.

In [35]:
# --- CELL 1: SETUP HỆ THỐNG & IMPORT ---
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cấu hình đường dẫn để Python nhìn thấy thư mục 'Module' (nằm ở Project Root)
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
# 2. Cấu hình để Python nhìn thấy file 'ChangePoint.py' (nằm cùng thư mục Notebook)
if current_dir not in sys.path:
    sys.path.append(current_dir)

print(f"Project Root: {project_root}")

# --- IMPORT MODULES ---
from Module.Load_Data import CleanDataLoader

# Import class chuẩn bị dữ liệu (Của Bạn)
from Module.ChangePoint import ChangePointPreparer

# Import class thuật toán (Của Nhóm Trưởng)
try:
    from Module.ChangePointDetector import ChangePointDetector
    print("✅ Đã import thành công: ChangePointDetector")
except ImportError:
    print("⚠️ Cảnh báo: Không tìm thấy 'ChangePointDetector' trong folder Module. Hãy kiểm tra lại file của nhóm trưởng.")

Project Root: c:\Users\lenovo\OneDrive\Document\PythonProject
⚠️ Cảnh báo: Không tìm thấy 'ChangePointDetector' trong folder Module. Hãy kiểm tra lại file của nhóm trưởng.


## 1. Chuẩn bị dữ liệu (Data Preparation)

In [36]:
# --- CELL 2: LẤY DỮ LIỆU TỪ CLEAN LOADER ---

# 1. Khởi tạo
preparer = ChangePointPreparer(clean_loader=CleanDataLoader())

# 2. Chọn đối tượng phân tích
# Môn học quan trọng
subjects_list = ["Toan", "Ngu_Van", "Ngoai_Ngu", "Vat_Li", "Lich_Su"]
# Khối thi quan trọng
blocks_list = ["A00", "A01", "B00", "C00", "D01"]
# Tỉnh thành lớn (Dùng tên không dấu viết liền theo quy ước)
provinces_list = ["HaNoi", "ThanhPhoHoChiMinh", "HaiPhong", "ĐaNang", "CanTho"]

# 3. Lấy chuỗi thời gian (Time Series)
print("⏳ Đang tải dữ liệu...")
df_subjects = preparer.get_subject_series(subjects_list, metric="mean")
df_blocks = preparer.get_block_series(blocks_list, metric="mean")
df_provinces = preparer.get_province_series(provinces_list, metric="mean")

print("\n--- Sample Data (Môn học) ---")
display(df_subjects)

print("\n--- Sample Data (Khối thi) ---")
display(df_blocks.head())

print("\n--- Sample Data (Tỉnh thành) ---")
display(df_provinces)

⏳ Đang tải dữ liệu...

--- Sample Data (Môn học) ---


,year,series_id,value
0,2023,Toan,6.250557
1,2024,Toan,6.447309
2,2025,Toan,4.799712
3,2023,Ngu_Van,6.858194
4,2024,Ngu_Van,7.231146
5,2025,Ngu_Van,6.990479
6,2023,Ngoai_Ngu,5.462410
7,2024,Ngoai_Ngu,5.521938
8,2025,Ngoai_Ngu,5.406338
9,2023,Vat_Li,6.573900



--- Sample Data (Khối thi) ---


,year,series_id,value
0,2023,A00,20.774499
1,2024,A00,20.904633
2,2025,A00,19.384368
3,2023,A01,20.275288
4,2024,A01,20.473793



--- Sample Data (Tỉnh thành) ---


,year,series_id,value
0,2023,HaNoi,37.324483
1,2024,HaNoi,39.081294
2,2025,HaNoi,25.724696
3,2023,ThanhPhoHoChiMinh,38.174897
4,2024,ThanhPhoHoChiMinh,38.443969
5,2025,ThanhPhoHoChiMinh,25.010852
6,2023,HaiPhong,39.020116
7,2024,HaiPhong,40.876944
8,2025,HaiPhong,25.183939
9,2023,ĐaNang,35.831963


## 2. Phát hiện điểm gãy (Change Point Detection)
Sử dụng các thuật toán: **PELT** (Ruptures) và **CUSUM**.

In [37]:
pip install ruptures

  Using cached ruptures-1.1.9.tar.gz (308 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build ruptures
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for ruptures (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [106 lines of output]
      C:\Users\lenovo\AppData\Local\Temp\pip-build-env-kseb9m4o\overlay\Lib\site-packages\setuptools_scm\_integration\deprecation.py:7: UserWarning: setup.cfg: at [metadata]
      version = attr: ... is forcing setuptools to override the version setuptools-scm did already set
      When using setuptools-scm it's invalid to use setuptools dynamic version as well, please remove it.
      Setuptools-scm is responsible for setting the version, forcing setuptools to override creates errors.
        warnings.warn(
      C:\Users\lenovo\AppData\Local\Temp\pip-build-env-kseb9m4o\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider remo

In [38]:
# --- CELL 3: CHẠY THUẬT TOÁN  ---

if 'ChangePointDetector' in globals():
    # 1. Khởi tạo bộ phát hiện (Giả thuyết điểm gãy là năm 2025, sai số 1 năm)
    # Chúng ta gộp cả 3 loại dữ liệu vào để chạy một thể cho hoành tráng
    full_data = pd.concat([df_subjects, df_blocks, df_provinces], ignore_index=True)
    
    detector = ChangePointDetector(full_data, target_year=2025, tolerance=1)

    # 2. Chạy PELT (Tinh chỉnh penalty=1 vì chuỗi ngắn 3 năm)
    print("\n▶️ Đang chạy thuật toán PELT...")
    df_pelt = detector.detect_ruptures(method="pelt", model="l2", pen=1)
    display(df_pelt.head())

    # 3. Chạy CUSUM
    print("\n▶️ Đang chạy thuật toán CUSUM...")
    df_cusum = detector.detect_cusum()
    display(df_cusum.head())

    # 4. Tổng hợp kết quả (Những series nào thực sự gãy ở 2025?)
    print("\n🏆 --- KẾT QUẢ PHÁT HIỆN ĐIỂM GÃY (HIT TARGET 2025) ---")
    all_results = detector.analyze_all()
    hits = all_results[all_results['hit_target'] == True]
    display(hits)
else:
    print("❌ Không thể chạy thuật toán vì thiếu Module ChangePointDetector.")

❌ Không thể chạy thuật toán vì thiếu Module ChangePointDetector.


## 3. Trực quan hóa (Visualization)
Vẽ biểu đồ minh họa sự thay đổi cấu trúc.

In [39]:
# --- CELL 4: VẼ BIỂU ĐỒ (PHIÊN BẢN GỠ LỖI) ---
# Giả sử detector đã chạy và có kết quả
if 'detector' in locals() and hasattr(detector, 'results'):
    # Lấy đại một kết quả có điểm gãy để vẽ demo
    # Ưu tiên lấy môn Toán hoặc Văn nếu có
    target_series = "Toan"
    row = detector.results[detector.results['series_id'] == target_series]
    
    if not row.empty and len(row.iloc[0]['detected_years']) > 0:
        years = row.iloc[0]['detected_years']
        print(f"Đang vẽ biểu đồ nâng cao cho: {target_series}")
        
        # GỌI HÀM MỚI TẠI ĐÂY
        detector.plot_enhanced(target_series, years)
    else:
        # Nếu Toán không gãy, lấy cái đầu tiên tìm được
        has_break = detector.results[detector.results['detected_years'].apply(len) > 0]
        if not has_break.empty:
            s_id = has_break.iloc[0]['series_id']
            yrs = has_break.iloc[0]['detected_years']
            detector.plot_enhanced(s_id, yrs)
        else:
            print("Chưa tìm thấy điểm gãy nào để vẽ.")
